# GridAPPS-D Python Library

---

## Intro to GridAPPSD-Python

GridAPPSD-Python is a Python library that can wrap API calls and pass them to the various GridAPPS-D APIs through the GOSS Message Bus.

The library has numerous shortcuts to help you develop applications faster and interface them with other applications, services, and GridAPPS-D compatible software packages.

The GridAPPSD-Python library requires a python version >= 3.6 and < 4 in order to work properly. (Note: no testing has been done with python 4 to date).


The GridAPPSD-Python library can be installed using `pip install gridappsd-python`.

For more information, see the GridAPPSD-Python [GitHub Repo](https://github.com/GRIDAPPSD/gridappsd-python/tree/poetry) and [PyPi site](https://pypi.org/project/gridappsd-python/).

---

## Connecting to GridAPPS-D Platform

Before starting any development in the GridAPPS-D environment, it is necessary to establish a connection to the GridAPPS-D Platform.


### Specifying Environment Variables (Preferred)

The preferred method for establishing a connection with the GridAPPS-D Platform is to define a set of environment variables that specify the connection address, port, username, and password. 



__Specifying the Environment Variables in Python Script__

This method is recommended for initial application development when running in a development environment, such as PyCharm or the Jupyter Notebook tutorials.

In [ ]:
# Establish connection to GridAPPS-D Platform:
from gridappsd import GridAPPSD

import os # Set username and password
os.environ['GRIDAPPSD_USER'] = 'tutorial_user'
os.environ['GRIDAPPSD_PASSWORD'] = '12345!'
os.environ['GRIDAPPSD_ADDRESS'] = 'localhost'
os.environ['GRIDAPPSD_PORT'] = '61613'

# Connect to GridAPPS-D Platform
gapps = GridAPPSD()
assert gapps.connected

__Specifying the Environment Variable in ~/.bashrc Script__

This method is recommended for more complete applications scripts where all the application scripts are called from a single ~/.bashrc script. In that script, the environment variables can be defined and then will be available to all scripts that need to connect the GridAPPS-D Platform.



```
# export allows all processes started by this shell to have access to the global variable

# address where the gridappsd server is running - default localhost
export GRIDAPPSD_ADDRESS=localhost

# port to connect to on the gridappsd server (the stomp client port)
export GRIDAPPSD_PORT=61613

# username to connect to the gridappsd server
export GRIDAPPSD_USER=app_user

# password to connect to the gridappsd server
export GRIDAPPSD_PASSWORD=1234App

# Note these should be changed on the server in a cyber secure environment!
```

---

### Specifying Connection Parameters Manually

An older method of connecting to the GridAPPS-D Platform is manually specifying the connection parameters. This method is still supported, but may be deprecated in future releases. 

This method is less flexible and has an in-built portability issues associated with hard-coded platform passwords.

In [ ]:
gapps = GridAPPSD("('localhost', 61613)", username='system', password='manager')

---

### GridAPPSD-utils Deprecated

GridAPPS-D Platform releases prior to 2021 used a library called `utils` to establish a connection with the platform. This library has been deprecated and replaced with Java Token Authentication using the environment variable method shown above.

The authentication method below will work with 2019-2020 versions of the GridAPPS-D Platform and GridAPPSD-Python, but not with any newer releases. 

```
# DEPRECATED authentication method
from gridappsd import GridAPPSD, utils
gapps = GridAPPSD(address=utils.get_gridappsd_address(),
          username=utils.get_gridappsd_user(), password=utils.get_gridappsd_pass())
```


`utils` -- __DEPRECATED__ A set of utilities to assist with common commands, inlcuding

 

<p style='text-align: left;'> Function Call | <p style='text-align: left;'> Usage
--------------|-----------
<p style='text-align: left;'> _utils.validate_gridappsd_uri()_ | <p style='text-align: left;'> Checks if GridAPPS-D is hosted on the correct port
<p style='text-align: left;' > _utils.get_gridappsd_address()_ | <p style='text-align: left;'> Returns the platform address such that response can be passed directly to a socket or the STOMP library
<p style='text-align: left;'> _utils.get_gridappsd_user()_ | <p style='text-align: left;'> Returns the login username 
<p style='text-align: left;'> _utils.get_gridappsd_pass()_ | <p style='text-align: left;'> Returns the login password
<p style='text-align: left;'> _utils.get_gridappsd_application_id()_ | <p style='text-align: left;'> Only applicable if the environment variable 'GRIDAPPSD_APPLICATION_ID' has been set
<p style='text-align: left;'> utils.get_gridappsd_simulation_id()_ | <p style='text-align: left;'> retrieves the simulation id from the environment.



__It is strongly recommended that applications that previously used this method replace any connection objects with environment variables to ensure compatibility with subsequent releases of the GRIDAPPS-D platform__

---

## Passing API calls with GridAPPSD-Python

There are three methods used in GridAPPSD-Python Library to pass API calls to the GridAPPS-D platform:

* `.get_response(self, topic, message, timeout)` -- Pass a database query, response expected before timeout
* `.subscribe(self, topic, callback)` -- Subscribe to a data stream
* `.send(self, topic, message)` -- Send a command to a simulation, no response expected

Each are explained in more detail below

### .get_response(topic, message)

This is the most commonly used method for passing API calls to the GridAPPS-D Platform. This method is used when a response is expected back from the GridAPPS-D platform within a particular timeout period. It is used for all database queries using


* [PowerGrid Models API](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb) -- queries for model info, object mRIDs, measurement mRIDs

* [Configuration File API](../api_usage/3.4-Using-the-Configuration-File-API.ipynb) -- queries to convert the model into other file format versions

* [Timeseries API](../api_usage/3.7-Using-the-Timeseries-API.ipynb) -- queries for weather data and historical data from prior simulations


The syntax used when calling this method is  `gapps.get_response(topic, message)` or alternatively, `gapps.get_response(topic, message, timeout = 30)`, where 

* `topic` is the GridAPPS-D topic for the particular API (as described in [API Communication Channels](../api_usage/3.1-API-Communication-Channels.ipynb).

* `message` is the query message specifying what information the API should return

* `timeout = ` is optional and gives the number of seconds given for the API to respond. Model conversion queries using the Configuration File API may take 30 - 60 seconds for very large models. Most other queries do not need a timeout specification.


---

### .subscribe(topic, message)

This method is used for subscribing to the real-time data stream generated by the GridAPPS-D platform while running a simulation. It is used to subscribe to information published at each time step by the 

* [Simulation API](../api_usage/3.6-Controlling-Simulation-API.ipynb) -- simulated SCADA data and measurements created by the simulation

* [Logging API](../api_usage/3.8-Using-the-Logging-API.ipynb) -- log messages published by the Platform, applications, and simulation

The `.subscribe()`method is also used to subscribe to streaming data generated by some of the GridAPPS-D services. 


The syntax used when calling this method is `gapps.subscribe(topic, message)`, where

* `topic` is the GridAPPS-D simulation output topic, log output topic, or service output topic for the particular real-time data stream the application needs to subscribe to, (as described in [API Communication Channels](../api_usage/3.1-API-Communication-Channels.ipynb).

* `message` is the subscription message. For simulation and log outputs, it is a method or class definition, as described in [Comparison of Subscription Approaches](/api_usage/3.6-Controlling-Simulation-API.ipynb#Comparison-of-Subscription-Approaches).

---

### .send(topic, message)

This method is used for sending equipment command and simulation input messages to the GridAPPS-D platform while running a simulation. It is used to send difference messages to the [Simulation API](../api_usage/3.6-Controlling-Simulation-API.ipynb) and for other generic publishing needs, such as sending a command input to a GridAPPS-D Service.

The syntax used when calling this method is `gapps.send(topic, message)`, where

* `topic` is the simulation or service input topic(as described in [API Communication Channels](../api_usage/3.1-API-Communication-Channels.ipynb).

* `message` is the API call message to be published. The most commonly used simulation input message is a [Difference Message](../api_usage/3.6-Controlling-Simulation-API.ipynb#Format-of-a-Difference-Message) used to control equipment settings.

---

### .unsubscribe(conn_id)

This method is used to unsubscribe from a simulation or service that was previously subscribed to using the `.subscribe` method. 

The syntax of this method is `gapps.unsubscribe(conn_id)`, where `conn_id` is the connection id obtained when previously subscribing using the `conn_id = gapps.subscribe(topic, message)`.

---

## Importing Required Python Libraries

A typical GridAPPS-D application will require several libraries to be imported from GridAPPSD-Python as well as from other Python libraries.


### Required GridAPPS-D Libraries

The GridAPPS-Python API contains several libraries, which are used to query for information, subscribe to measurements, and publish commands to the GOSS message bus. These inlcude


* `GridAPPSD` -- This is the primary library that contains numerous methods for passing API calls, connecting to the GridAPPS-D platform, and other common tasks

* `topics` -- This library contains methods for constructing the correct API channel strings

* `Simulation` -- This library contains shortcut methods for subscribing and controlling simulations

* `Logger` -- This library contains logging methods. It is recommended to invoke those methods using the `gapps.get_logger` method rather than importing the library

* `GOSS` -- This library contains methods for passing API calls to the GOSS Message Bus. It is imported automatically when importing the `GridAPPSD` library

* `Houses` -- This library populates a feeder with thermal house model loads. It is imported automatically when importing the `GridAPPS` library 

* `utils` -- Deprecated


Each of the libraries can be imported using `from gridappsd import library_name`. For example,

In [ ]:
from gridappsd import GridAPPSD

In [ ]:
from gridappsd import topics as t

Each of the libraries are discussed in detail in the next section.

---

### Other Required Python Libraries

Below is a list of some of the additional libraries that you may need to import. 

You may not need all of these additional libraries, depending on the needs of your application

* `argparse` -- This is the recommended command-line parsing module in Python.([Online Documentation](https://docs.python.org/3/howto/argparse.html))

* `json` -- Encoder and decoder for JavaScript Object Notation (JSON). ([Online Documentation](https://docs.python.org/3/library/json.html))

* `logging` -- This module defines classes and functions for event logging. ([Online Documentation](https://docs.python.org/3/library/logging.html)

* `sys` -- Python module for system specific parameters. ([Online Documentation](https://docs.python.org/3/library/sys.html))

* `time` -- Time access and conversions. ([Online Documentation](https://docs.python.org/3/library/time.html))

* `pytz` -- Library to enable resolution of cross-platform time zones and ambiguous times. ([Online Documentation](https://pypi.org/project/pytz/)

* `stomp` -- Python client for accessing messaging servers using the Simple Text Oriented Messaging Protocol (STOMP). ([Online Documentation](https://pypi.org/project/stomp.py/))

* `os` -- Miscellaneous operating system interface. Needed to set environment variables for the GridAPPS-D connection if working from a single Python script or notebook. ([Online Documentation](https://docs.python.org/3/library/os.html))


In [ ]:
import argparse
import json
import logging
import sys
import time
import pytz
import stomp
import os

---

## GridAPPSD-Python GridAPPSD Library

This library contains the most commonly used methods needed for building GridAPPS-D applications and services.

All of these methods are for the GridAPPS-D connection object defined using `gapps = GridAPPSD()`

### Get Methods


This group of methods are used to get information and statuses about the GridAPPS-D platform and simulations:


* `.get_application_status()` -- Returns the current status of an application

* `.get_application_id()` -- Returns the unique ID of an application registered with the Platform

* `.get_houses()` -- Returns houses populated in the feeder

* `.get_logger()` -- Returns a log instance for interacting with logs within the Platform

* `.get_platform_status()` -- Returns the current status of the Platform

* `.get_service_status()` -- Returns the current status of a service
 
* `.get_simulation_id()` -- Returns the simulation ID for the current GridAPPSD connection


### Set / Send Methods

This group of methods are used to set the status of applications and services:


* `.set_application_status()` -- Set the status of an application

* `.set_service_status()` -- Set the status of a service

* `.set_simulation_id(simulation_id)` -- Set the simulation ID if none is defined

* `.send_simulation_status(status, message, log_level)` -- Sets simulation + service status and writes to GridAPPS-D logs

* `.send_status(status, message, log_level)` -- Sets application status and writes to GridAPPS-D logs

### PowerGrid Models API Methods

This group of methods run pre-built PowerGrid Models API queries for simpler query types:

* `query_data(query, timeout)` -- [Run a generic SPARQL Query](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb#Query-using-a-Generic-SPARQL-Query)

* `query_model(model_id, object_type, object_id)` -- [Query using full CIM100 prefix](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb#Query-using-a-SPARQL-filter)

* `query_model_info()` -- [Query for dictionary of all models](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb#Query-for-Dictionary-of-all-Models)

* `query_model_names(model_id)` -- [Query for mRIDs of all models](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb#Query-for-Dictionary-of-all-Models)

* `query_object(object_id, model_id)` -- [Query for CIM attributes of an object](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb#Query-for-CIM-Attributes-of-an-Object)

* `query_object_dictionary(model_id, object_type, object_id)` -- [Query for object dictionary](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb#Query-for-Object-Dictionary)

* `query_object_types(model_id)` -- [Query for CIM classes in a model](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb#Query-for-Dictionary-of-all-Models)

---

## GridAPPSD-Python Topics Library

The GridAPPSD-Python topics library is used to obtain the correct [API Communication Channel](../api_usage/3.1-API-Communication-Channels.ipynb), which tells the GridAPPS-D platform to which database, application, or simulation a particular API call should be delivered. 

Static GridAPPS-D topics (such as those for the [PowerGrid Models API](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb), [Configuration File API](../api_usage/3.4-Using-the-Configuration-File-API.ipynb), and [Timeseries API](../api_usage/3.7-Using-the-Timeseries-API.ipynb)) can be imported using

In [ ]:
from gridappsd import topics as t

Dynamic GridAPPS-D topics (such as those for the [Simulation API](../api_usage/3.6-Controlling-Simulation-API.ipynb) and various GridAPPS-D services) can be imported using

In [ ]:
from gridappsd.topics import simulation_output_topic

In [ ]:
from gridappsd.topics import simulation_input_topic

In [ ]:
from gridappsd.topics import simulation_log_topic

Each of the specific methods available in the `topics` library are discussed in detail in [API Communication Channels](../api_usage/3.1-API-Communication-Channels.ipynb).

---

---

## GridAPPSD-Python Simulation Library

### GridAPPSD-Python Simulation Library

The GridAPPSD-Python simulation library is used for starting, running, and controlling parallel digital twin simulations. For more details on specific usage, see


* [Starting a Parallel Simulation](../api_usage/3.5-Creating-Running-Simulation-API.ipynb#Starting-the-Simulation)

* [Pausing, Resuming, and Stopping Parallel Simulations](../api_usage/3.5-Creating-Running-Simulation-API.ipynb#Using-the-gridappsd.simulation-Python-Library)

* [Subscribing to Parallel Simulations](../api_usage/3.6-Controlling-Simulation-API.ipynb#Subscribing-to-Parallel-Simulations)



The Simulation library can be imported using

In [ ]:
from gridappsd.simulation import Simulation

Available methods in the `Simulation` library are

* `.start_simulation()` -- Start the simulation


* `.pause()` -- Pause the simulation


* `.resume()` -- Resume the simulation


* `.resume_pause_at(pause_time)` -- Resume the simulation, and then pause it in so many seconds


* `.stop()` -- Stop the simulation


* `.run_loop()` -- Loop the entire simulation until interrupted


* `.simulation_id` -- Returns the Simulation ID of the simulation


* `.add_ontimestep_callback(myfunction1)` -- Run the desired function on each timestep


* `.add_onmesurement_callback(myfunction2)` -- Run the desired function when a measurement is received.


* `.add_oncomplete_callback(myfunction3)` -- Run the desired function when simulation is finished


* `.add_onstart_callback(myfunction4)` -- Run desired function when simulation is started


__Note: method name `.add_onmesurement` is misspelled in the library definition!!__


---

### GridAPPSD-Python DifferenceBuilder

`DifferenceBuilder` is a GridAPPSD-Python library that is used to create and correctly format difference messages that used to create equipment control commands. The usage of difference builder is given in [Using DifferenceBuilder](../api_usage/3.6-Controlling-Simulation-API.ipynb#Using-GridAPPSD-Python-DifferenceBuilder).

The `DifferenceBuilder` library can be imported using

In [ ]:
from gridappsd import DifferenceBuilder

my_diff_build = DifferenceBuilder(simulation_id)

---

[Return to Top](#Table-of-Contents)

---

---

![gridappsd-logo](../images/GridAPPS-D_narrow.png)